In [75]:
!sudo apt-get update
!apt-get install poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,747 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,253 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9

In [1]:
!pip install pdfplumber spacy pdf2image mistralai pydantic pytesseract asyncio pymupdf nest_asyncio

##Testing

Do not run when executing main workflow

In [2]:
import pdfplumber
from google import genai
import io
import httpx
from google.colab import userdata
import pathlib
import types
import base64
import os
from mistralai import Mistral
import asyncio
from pydantic import BaseModel
import fitz
import nest_asyncio
from IPython.display import display
import difflib

In [ ]:
import cv2
from PIL import Image, ImageEnhance, ImageFilter
# grayscale
img = Image.open("page_0.png").convert("L")
MAX_DIM = 2000
if img.width > MAX_DIM or img.height > MAX_DIM:
    scale = MAX_DIM / max(img.width, img.height)
    img = img.resize((int(img.width * scale), int(img.height * scale)), Image.BICUBIC)

img = ImageEnhance.Contrast(img).enhance(2.0)

img = img.filter(ImageFilter.MedianFilter(size=3))
img = img.resize((img.width * 2, img.height * 2), Image.BICUBIC)

img.save("soft_cleaned_fax_safe.png")

from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

# Load TrOCR
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

image = Image.open("soft_cleaned_fax_safe.png").convert("RGB")
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values, max_length=256)
text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("OCR result:", text)


#Referral Extraction

In [3]:
page_num = 0
with pdfplumber.open("referral_package.pdf") as pdf:
    page_num = len(pdf.pages)

print(page_num)

15


In [4]:
def page_to_img(path, page_number):
    with pdfplumber.open(path) as pdf:
        image = pdf.pages[page_number].to_image(resolution=150).original  # PIL Image
        filename = f"page_{page_number}.png"
        image.save(filename)
        return filename  # Return the filename

In [5]:
def process_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

In [6]:
#Testing on handwriting page
image_path = "page_0.png"
base64_image = process_image(image_path)

api_key = userdata.get("MISTRALAI_API_KEY")
client = Mistral(api_key=api_key)



In [7]:
def encode_pdf(pdf_path):
    """Encode the pdf to base64."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {pdf_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

base64_pdf = encode_pdf("referral_package.pdf")

test_response2 = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": f"data:application/pdf;base64,{base64_pdf}"
    },
    include_image_base64=True
)

In [8]:
test_response2

OCRResponse(pages=[OCRPageObject(index=0, markdown='04/22/2024 WED 11:32 FAX\n\n001/028\n\n# FAX TRANSMISSION\n\n**Better Life Multiple Sclerosis Center**\n\n3320 Montgomery Dr. Nashville, TN 37361\n\n**BetterLife**\n\nF 615-562-4820 P: 615-562-4848\n\nDr. Asriel Han | Dr. Aetya Shan\n\n---\n\n**TO:**\n\nGolden Gate Infusion Center\n\n**Fax:** 614-225-3355 **Phone:** 614-295-7655\n\n**From:** Erran Rostami, BSN, RN\n\n**P:** 615-343-1176\n\n**F:** 615-343-1219\n\n---\n\n**Page**\n\n(including cover sheet)\n\n---\n\n**Comments:**\n\n- Arabic - spoken / English - written\n- Rituxen (Truxima) TP\n- MRI Reports\n- Hospital DIC Make\n- Demographics\n\n---\n\nThe documents accompanying this transmission may contain health information that is legally protected. This information is intended only for the use of the individual or entity named above. The authorized recipient of this information is prohibited from disclosing this information to any other party unless permitted by law or regulation

In [9]:
def extract_page_contents(ocr_response):
    pages = ocr_response.pages
    return [
        {
            "page": page.index,
            "page_content": page.markdown.strip() if hasattr(page, "markdown") else ""
        }
        for page in pages
    ]


In [10]:
referral_pack_response = extract_page_contents(test_response2)

In [11]:
referral_pack_response

[{'page': 0,
  'page_content': '04/22/2024 WED 11:32 FAX\n\n001/028\n\n# FAX TRANSMISSION\n\n**Better Life Multiple Sclerosis Center**\n\n3320 Montgomery Dr. Nashville, TN 37361\n\n**BetterLife**\n\nF 615-562-4820 P: 615-562-4848\n\nDr. Asriel Han | Dr. Aetya Shan\n\n---\n\n**TO:**\n\nGolden Gate Infusion Center\n\n**Fax:** 614-225-3355 **Phone:** 614-295-7655\n\n**From:** Erran Rostami, BSN, RN\n\n**P:** 615-343-1176\n\n**F:** 615-343-1219\n\n---\n\n**Page**\n\n(including cover sheet)\n\n---\n\n**Comments:**\n\n- Arabic - spoken / English - written\n- Rituxen (Truxima) TP\n- MRI Reports\n- Hospital DIC Make\n- Demographics\n\n---\n\nThe documents accompanying this transmission may contain health information that is legally protected. This information is intended only for the use of the individual or entity named above. The authorized recipient of this information is prohibited from disclosing this information to any other party unless permitted by law or regulation.\n\nIf you are not 

In [12]:
import pytesseract
from PIL import Image

In [13]:
img = Image.open("page_14.png")
#TEST
# Run OCR with Tesseract or any other visual OCR engine
text = pytesseract.image_to_string(img)

print(text)

05/22/2024 WED 11:35 FAX MIVLI/V29
5/22/24, 10:39 AM

ae : , cs « ‘a
predniSONE 20 mg tablet Taka 3 tablets (60 mg total) by mouth daily for 7

Commonly known as: DELTASONE days, THEN 2 tablets (40 mg total) daily for 7

Start taking on: May'16, 2024 days, THEN 1 tablet (20 mg total) dally for 7
days, THEN 0,5 tablets (10 mg total) daily for 7
days, Then stop.

Activity instructions

Discharge activity - Normal Activity

Some medications such as for example pain medications and sleeping or anti-anxiety agents may affect your
abllity to operate heavy machinery including driving. Please always carefully read the labels on the medications
that you take, If you have experienced confusion, seizures, lightheadedness or blackouts then it may not be safe
or even Jegal for you to operate heavy machinery (Including driving). Please act in the interest of your own safety
and that of others. Speak to a physician If you ara not sure whether it Ie safe or legal for you to drive,
Care Instructions: Y

In [14]:
def extract_page_contents_fallback(pages):
  pages_content = []
  for i in range(pages):
    img = Image.open(f"page_{i}.png")
    text = pytesseract.image_to_string(img)
    content = {
        "page": i,
        "page_content": text
    }
    pages_content.append(content)
  return pages_content

In [15]:
pytes_results = extract_page_contents_fallback(page_num)
pytes_results

[{'page': 0,
  'page_content': "04/22/2024 WRD 11:32 FAX KIVULAU Sd\n\nFAX TRANSMISSION\n\nBetter Lie Multiple Sclerosis Center\n3320 Montgomery Dr> Nashville, TN 37364\nF 618-662-4820 P: 615-562-4848\nDr. Asriel Han | Dr. Aditya Shan\n\nBetterLife\n\nTO: Golden Gate Infusion Center\n\nFax: ~G/4—-2-75-5 35S Phone Orazge 57 ONT\n\nFrom: Erfan Rostami, BSN, RN\n\nPages P: 618-343-1176\n\n(including cover sheet)\nF: 615-343-1219\n\nComments:\n“7 OS ) . +r\n(Arav'e — Seoten / Sng bsin ~ aa ~)\n7 i baxter (Truse, ma ) TY?\n\n~ mee pout\n- Hos | dé cM be\n\n- Demonnogh« 9\n\nThe documents accompanying this transmission may contain health information that Is legally protected. This information ia intanded only\nfor the use of the Individual or entity named above, The authorized recipient of this information ts prohibited from disclosing this\nInformation to any other party unless permitted by law or regulation.\n\nIf you ara not the intended recipient, you are hereby notified that any use, dl

In [16]:
client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

In [21]:
def evaluate_page(page_num, mistral_text, pytess_text):
    prompt = f"""
You are comparing two OCR outputs from page {page_num} of a medical document.

Choose the output that is:
- More accurate and complete
- Less likely to include hallucinated or repeated content
- Reflective of a medical fax or discharge summary

Do NOT answer with just "A" or "B".
Do NOT describe your reasoning.
Instead, return the full content of the better OCR output below.

### OCR Output A (Mistral):
{mistral_text}

### OCR Output B (Tesseract):
{pytess_text}

Respond with a strict JSON object ONLY, containing the better content:
{{
  "page": {page_num},
  "content": "<<<full better OCR text here>>>"
}}
No markdown, no extra text — just JSON.
"""

    response = client.models.generate_content(
        model="models/gemini-2.0-flash",
        contents=prompt
    )

    return response.text if hasattr(response, "text") else str(response)


import asyncio
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor()

async def evaluate_all_pages(mistral_pages, pytess_pages):
    loop = asyncio.get_event_loop()
    tasks = []
    for i in range(len(mistral_pages)):
        m_text = mistral_pages[i]["page_content"]
        p_text = pytess_pages[i]["page_content"]
        tasks.append(loop.run_in_executor(executor, evaluate_page, i, m_text, p_text))
    return await asyncio.gather(*tasks)



/usr/lib/python3.11/tokenize.py:529: RuntimeWarning: coroutine 'evaluate_all_pages' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)


In [23]:
results = await evaluate_all_pages(referral_pack_response, pytes_results)

In [24]:
def extract_clean_json(responses):
    import re
    import json

    parsed = []
    for r in responses:
        # Remove markdown-style formatting
        cleaned = re.sub(r"^```json|```$", "", r.strip(), flags=re.MULTILINE).strip()

        # Try parsing it
        try:
            parsed.append(json.loads(cleaned))
        except Exception as e:
            print(f"Failed to parse JSON: {e}")
            parsed.append(None)
    return parsed

parsed_results = extract_clean_json(results)
parsed_results

[{'page': 0,
  'content': '04/22/2024 WED 11:32 FAX\n\n001/028\n\n# FAX TRANSMISSION\n\n**Better Life Multiple Sclerosis Center**\n\n3320 Montgomery Dr. Nashville, TN 37361\n\n**BetterLife**\n\nF 615-562-4820 P: 615-562-4848\n\nDr. Asriel Han | Dr. Aetya Shan\n\n---\n\n**TO:**\n\nGolden Gate Infusion Center\n\n**Fax:** 614-225-3355 **Phone:** 614-295-7655\n\n**From:** Erran Rostami, BSN, RN\n\n**P:** 615-343-1176\n\n**F:** 615-343-1219\n\n---\n\n**Page**\n\n(including cover sheet)\n\n---\n\n**Comments:**\n\n- Arabic - spoken / English - written\n- Rituxen (Truxima) TP\n- MRI Reports\n- Hospital DIC Make\n- Demographics\n\n---\n\nThe documents accompanying this transmission may contain health information that is legally protected. This information is intended only for the use of the individual or entity named above. The authorized recipient of this information is prohibited from disclosing this information to any other party unless permitted by law or regulation.\n\nIf you are not the i

In [51]:
pa_summary_prompt = """
You are a medical document specialist preparing a Prior Authorization (PA) summary.

Your task is to review the full content of a referral packet—including faxes, therapy plans, and clinical notes—and generate a **concise PA summary report** that contains information typically required for prior authorization decisions.

Organize the summary using the following sections (include only if information is found):

1. **Patient Information** – name, DOB, contact info, address, MRN
2. **Referring Provider** – name, practice name, phone, fax
3. **Requesting Nurse/Coordinator** – name, phone, fax (if applicable)
4. **Diagnosis and Clinical Details** – primary diagnosis (ICD-10 if found), clinical history, MS/cancer subtype, other relevant conditions
5. **Treatment Plan** – medication, dosage, schedule, administration site, start date, maintenance plan
6. **Pre-Medications & Infusion Orders** – include key pre-meds and infusion monitoring instructions
7. **Labs & Monitoring** – labs to be collected, frequency
8. **PRN Medications** – rescue medications ordered
9. **Coverage Notes** – biosimilar use, contraindications, reasons for preferred drug, logistical constraints

Summarize clearly and professionally. Use bullet points or short paragraphs. Do not hallucinate. Do not invent information. If something is mentioned but unclear, write “not specified” or “unclear.”

Here is the full content of the document:
"""

In [52]:
import asyncio
import json
from typing import List, Dict, Any
from concurrent.futures import ThreadPoolExecutor

MODEL = "models/gemini-2.0-flash"
executor = ThreadPoolExecutor(max_workers=8)  # Adjust based on system resources

async def generate_pa_summary(pages: List[Dict[str, Any]], client) -> str:
    try:
        full_text = "\n\n---\n\n".join(
            f"Page {page.get('page', idx)}:\n{page['content']}"
            for idx, page in enumerate(pages)
        )

        prompt = pa_summary_prompt + full_text

        loop = asyncio.get_event_loop()
        response = await loop.run_in_executor(
            executor,
            lambda: client.models.generate_content(
                model=MODEL,
                contents=[prompt]
            )
        )

        return response.candidates[0].content.parts[0].text

    except Exception as e:
        return f"[Error during PA summary generation]: {str(e)}"




In [53]:
import json

In [54]:
summary_text = await generate_pa_summary(parsed_results, client)
print(summary_text)

**PA Summary Report**

1.  **Patient Information**
    *   Name: Shakh Abdulla
    *   DOB: 04/01/2001
    *   MRN: 041152153
    *   Address: 425 Sherman Ave, Nashville TN 37995 (Preferred address)
    *   Phone: 865-395-3958 (Home), 865-395-0481 (Mobile), 865-839-7458 (alternate)

2.  **Referring Provider**
    *   Name: Hao H Gu, MD
    *   Practice: Better Life Multiple Sclerosis Center
    *   Phone: 614-936-0065
    *   Fax: Not specified

3.  **Requesting Nurse/Coordinator**
    *   Name: Erfan Rostami, BSN, RN
    *   Phone: 615-343-1176
    *   Fax: 615-343-1219

4.  **Diagnosis and Clinical Details**
    *   Primary Diagnosis: Multiple sclerosis in pediatric patient (CMS/HCC) \[G35]
    *   Clinical History:
        *   Optic neuritis since October 2023.
        *   Right eye pain with movement.
        *   Visual field deficits.
        *   Numbness in right hand, muscle twitch, urinary hesitancy/urgency since April 2024.
        *   Positive ANA and SSB.
    *   MRI brain a

#PA Extraction

In [55]:
doc = fitz.open(("PA.pdf"))

doc[1].get_text('blocks')

[(18.0,
  768.580810546875,
  87.8330078125,
  779.4735107421875,
  'GR-68535-3 (1-25) \n',
  0,
  0),
 (161.99952697753906,
  10.737120628356934,
  427.83404541015625,
  108.46498107910156,
  'MEDICARE FORM \nRiabni\n® (rituximab-arrx),  \nRituxan\n® (rituximab), Ruxience  \n(rituximab-pvvr), Truxima (rituximab-abbs) \nMedication Precertification Request \nPage 2 of 5 \n(All fields must be completed and return both pages for precertification review.) \n',
  1,
  0),
 (453.7799987792969,
  13.48285961151123,
  585.0285034179688,
  44.80995559692383,
  'For Medicare Advantage Part B: \nFor other lines of business:  \nPlease use commercial form. \n',
  2,
  0),
 (454.55999755859375,
  47.52688217163086,
  589.5609130859375,
  89.79430389404297,
  'Note: Riabni and Rituxan are non-\npreferred. The preferred biosimilar \nproducts are Ruxience and Truxima.  \nFor rheumatoid arthritis, all Rituxan and \nbiosimilar products are non-preferred. \n',
  3,
  0),
 (77.98930358886719,
  110.7192153

In [56]:
def get_blocks(pdf_path):
  doc = fitz.open(("PA.pdf"))
  block_list = []
  for i, p in enumerate(doc, start=1):
    blocks = p.get_text('blocks')
    for block in blocks:
      obj = {
          "page": i,
          "block": block
      }
      print(obj)
      block_list.append(obj)
  return block_list

blocks = get_blocks("PA.pdf")
print(blocks)
#{'page': 2, 'name': 'T12', 'type': 7, 'rect': [62.843101501464844, 150.93402099609375, 230.375, 162.61297607421875], 'label': 'Patient First Name'}

{'page': 1, 'block': (18.0, 768.580810546875, 87.8329849243164, 779.4735107421875, 'GR-68535-3 (1-25) \n', 0, 0)}
{'page': 1, 'block': (171.1199951171875, 88.86286163330078, 218.37191772460938, 101.06350708007812, 'Page 1 of 5 \n', 1, 0)}
{'page': 1, 'block': (171.11952209472656, 11.582880020141602, 436.9670715332031, 92.01454162597656, 'MEDICARE FORM \nRiabni\n® (rituximab-arrx),  \nRituxan\n® (rituximab), Ruxience \n(rituximab-pvvr), Truxima (rituximab-abbs) \nMedication Precertification Request \n', 2, 0)}
{'page': 1, 'block': (445.3800048828125, 14.48285961151123, 576.630615234375, 45.81087112426758, 'For Medicare Advantage Part B: \nFor other lines of business:  \nPlease use commercial form. \n', 3, 0)}
{'page': 1, 'block': (445.3800048828125, 48.52688217163086, 580.3668823242188, 91.36349487304688, 'Note: Riabni and Rituxan are non-\npreferred. The preferred biosimilar \nproducts are Ruxience and Truxima.  \nFor rheumatoid arthritis, all Rituxan and \nbiosimilar products are non-

In [58]:
def extract_pdf_fields(pdf_path):
    doc = fitz.open(pdf_path)
    fields = []

    for i, page in enumerate(doc, start=1):
        block = page.get_text('blocks')
        for w in page.widgets():
            field = {
                "page": i,
                "name": w.field_name,
                "type": w.field_type,
                "rect": list(w.rect),  # Convert to list for JSON compatibility
                "label": w.field_label
            }
            fields.append(field)
    for f in fields:
      print(f)

    return fields

fields = extract_pdf_fields("PA.pdf")
print(fields)

{'page': 2, 'name': 'CB1', 'type': 2, 'rect': [82.92900085449219, 110.4580078125, 93.23079681396484, 121.0150146484375], 'label': 'Start of treatment'}
{'page': 2, 'name': 'T2', 'type': 7, 'rect': [210.19700622558594, 109.281982421875, 231.8699951171875, 120.96099853515625], 'label': 'Start date: (MM)'}
{'page': 2, 'name': 'T3', 'type': 7, 'rect': [236.83399963378906, 109.281982421875, 258.5069885253906, 120.96099853515625], 'label': 'Start date: (DD)'}
{'page': 2, 'name': 'T4', 'type': 7, 'rect': [262.1719970703125, 109.281982421875, 285.8070068359375, 120.96099853515625], 'label': 'Start date: (YYYY)'}
{'page': 2, 'name': 'CB5', 'type': 2, 'rect': [306.0480041503906, 110.03802490234375, 316.3500061035156, 120.594970703125], 'label': 'Continuation of therapy'}
{'page': 2, 'name': 'T6', 'type': 7, 'rect': [502.3739929199219, 109.2760009765625, 524.0469970703125, 120.95501708984375], 'label': 'Date of last treatment: (MM)'}
{'page': 2, 'name': 'T7', 'type': 7, 'rect': [529.010986328125,

In [59]:
client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

In [60]:
def get_element_by_value(key, value, array):
  filter = [e for e in array if e[key] == 2]
  return filter

In [61]:
print(get_element_by_value("page", 2, fields))

[{'page': 2, 'name': 'CB1', 'type': 2, 'rect': [82.92900085449219, 110.4580078125, 93.23079681396484, 121.0150146484375], 'label': 'Start of treatment'}, {'page': 2, 'name': 'T2', 'type': 7, 'rect': [210.19700622558594, 109.281982421875, 231.8699951171875, 120.96099853515625], 'label': 'Start date: (MM)'}, {'page': 2, 'name': 'T3', 'type': 7, 'rect': [236.83399963378906, 109.281982421875, 258.5069885253906, 120.96099853515625], 'label': 'Start date: (DD)'}, {'page': 2, 'name': 'T4', 'type': 7, 'rect': [262.1719970703125, 109.281982421875, 285.8070068359375, 120.96099853515625], 'label': 'Start date: (YYYY)'}, {'page': 2, 'name': 'CB5', 'type': 2, 'rect': [306.0480041503906, 110.03802490234375, 316.3500061035156, 120.594970703125], 'label': 'Continuation of therapy'}, {'page': 2, 'name': 'T6', 'type': 7, 'rect': [502.3739929199219, 109.2760009765625, 524.0469970703125, 120.95501708984375], 'label': 'Date of last treatment: (MM)'}, {'page': 2, 'name': 'T7', 'type': 7, 'rect': [529.010986

In [62]:
print(get_element_by_value("page", 2, blocks))

[{'page': 2, 'block': (18.0, 768.580810546875, 87.8330078125, 779.4735107421875, 'GR-68535-3 (1-25) \n', 0, 0)}, {'page': 2, 'block': (161.99952697753906, 10.737120628356934, 427.83404541015625, 108.46498107910156, 'MEDICARE FORM \nRiabni\n® (rituximab-arrx),  \nRituxan\n® (rituximab), Ruxience  \n(rituximab-pvvr), Truxima (rituximab-abbs) \nMedication Precertification Request \nPage 2 of 5 \n(All fields must be completed and return both pages for precertification review.) \n', 1, 0)}, {'page': 2, 'block': (453.7799987792969, 13.48285961151123, 585.0285034179688, 44.80995559692383, 'For Medicare Advantage Part B: \nFor other lines of business:  \nPlease use commercial form. \n', 2, 0)}, {'page': 2, 'block': (454.55999755859375, 47.52688217163086, 589.5609130859375, 89.79430389404297, 'Note: Riabni and Rituxan are non-\npreferred. The preferred biosimilar \nproducts are Ruxience and Truxima.  \nFor rheumatoid arthritis, all Rituxan and \nbiosimilar products are non-preferred. \n', 3, 0)

In [64]:
def rects_intersect(rect1, rect2):
    x0, y0, x1, y1 = rect1
    a0, b0, a1, b1 = rect2
    dx = max(0, min(x1, a1) - max(x0, a0))
    dy = max(0, min(y1, b1) - max(y0, b0))
    return dx * dy > 0

def rects_intersect_debug(rect1, rect2):
    x0, y0, x1, y1 = rect1
    a0, b0, a1, b1 = rect2
    dx = min(x1, a1) - max(x0, a0)
    dy = min(y1, b1) - max(y0, b0)
    intersection = max(0, dx) * max(0, dy)
    #print(f"[DEBUG] Checking overlap between {rect1} and {rect2}: dx={dx:.3f}, dy={dy:.3f}, intersection={intersection:.3f}")
    return intersection > 0

def text_similarity(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, a.lower(), b.lower()).ratio()

def map_fields_to_blocks(fields, blocks):
    labeled_fields = []

    for field in fields:
        field_rect = field["rect"]
        field_page = field["page"]
        field_label = field["label"]
        field_label_lower = field_label.lower()

        #print(f"\n[INFO] Processing field '{field_label}' on page {field_page} with rect {field_rect}")

        overlapping_blocks = []

        for block in blocks:
            if block["page"] != field_page:
                continue

            block_rect = block["block"][:4]

            if rects_intersect_debug(field_rect, block_rect):
                overlapping_blocks.append(block)

        best_block_context = ""

        if not overlapping_blocks:
            print("[INFO] No overlapping blocks found for this field.")
        else:
            # First fallback: substring match for short labels like "yes", "no", etc.
            for block in overlapping_blocks:
                block_text = block["block"][4].lower()
                if field_label_lower in block_text:
                    best_block_context = block["block"][4]
                    #print(f"[INFO] Substring fallback matched '{field_label}' in block text.")
                    break

            # If no substring match found, use text similarity
            if not best_block_context:
                best_score = 0
                for block in overlapping_blocks:
                    block_text = block["block"][4]
                    score = text_similarity(field_label, block_text)
                    #print(f"  [DEBUG] Similarity between field label '{field_label}' and block text start '{block_text[:30]}...': {score:.3f}")

                    if score > best_score:
                        best_score = score
                        best_block_context = block_text

                #print(f"[INFO] Best matching block context selected with similarity score {best_score:.3f}")

        labeled_fields.append({
            "page": field_page,
            "name": field["name"],
            "type": field["type"],
            "rect": list(field_rect),
            "label": field_label,
            "block_context": best_block_context
        })

        print({
            "page": field_page,
            "name": field["name"],
            "type": field["type"],
            "rect": list(field_rect),
            "label": field_label,
            "block_context": best_block_context
        })

    return labeled_fields


# Example usage:
# labeled_blocks = map_fields_to_blocks(fields, blocks)
# print(labeled_blocks)


labeled_blocks = map_fields_to_blocks(fields, blocks)

{'page': 2, 'name': 'CB1', 'type': 2, 'rect': [82.92900085449219, 110.4580078125, 93.23079681396484, 121.0150146484375], 'label': 'Start of treatment', 'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n'}
{'page': 2, 'name': 'T2', 'type': 7, 'rect': [210.19700622558594, 109.281982421875, 231.8699951171875, 120.96099853515625], 'label': 'Start date: (MM)', 'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n'}
{'page': 2, 'name': 'T3', 'type': 7, 'rect': [236.83399963378906, 109.281982421875, 258.5069885253906, 120.96099853515625], 'label': 'Start date: (DD)', 'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n'}
{'page': 2, 'name': 'T4', 'type': 7, 'rect': [262.1719970703125, 109.281982421875, 285.8070068359375, 120.96099853515625], 'label': 'Start date: (YYYY)', 'block_

# Filling the Document


In [95]:
def generate_field_prompt(block_context: str, referral_summary: str, label: str) -> str:
    return (
        f"You are helping to fill out a prior authorization (PA) form.\n\n"
        f"### Referral Summary:\n{referral_summary}\n\n"
        f"### Form Field Context:\n{block_context}\n\n"
        f"### Label:\n{label}\n\n"
        f"Based on the above, provide a clear and accurate answer to this field.\n"
        f"If a specific medication, date, diagnosis, or yes/no is requested, extract it directly from the summary.\n"
        f"If a label asks YYYY, only put in the year; if it asks for MM, only enter the month; and if it asks for DD only enter the day.\n"
        f"A single block context can have multiple fields, such as yes/no, multiple yes/nos, multiple fill-in lines, "
        f"DD/MM/YYYY fields, or a combination of any or all field types mentioned beforehand."
        f"If you determine that a field does not need to be filled, do not return an answer at all"
    )

In [100]:
from asyncio import Semaphore
import random

In [101]:
MODEL = "models/gemini-2.0-flash"
MAX_RETRIES = 5
RETRY_BASE_DELAY = 5
semaphore = Semaphore(3)  # Limit to 3 concurrent LLM calls

async def fill_single_field(field: Dict[str, Any], referral_summary: str, client) -> Dict[str, Any]:
    prompt = generate_field_prompt(
        block_context=field["block_context"],
        referral_summary=referral_summary,
        label=field["label"]
    )

    async with semaphore:
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                loop = asyncio.get_event_loop()
                response = await loop.run_in_executor(
                    None,
                    lambda: client.models.generate_content(
                        model=MODEL,
                        contents=prompt
                    )
                )
                answer = response.candidates[0].content.parts[0].text.strip()
                return {**field, "answer": answer}
            except Exception as e:
                error_msg = str(e)
                if "RESOURCE_EXHAUSTED" in error_msg or "429" in error_msg:
                    wait_time = RETRY_BASE_DELAY * (2 ** (attempt - 1)) + random.uniform(0, 1)
                    print(f"Rate limit hit. Retrying field '{field['name']}' in {wait_time:.2f}s (attempt {attempt}/{MAX_RETRIES})...")
                    await asyncio.sleep(wait_time)
                else:
                    return {**field, "answer": f"Error: {error_msg}"}

        return {**field, "answer": f"Error: Max retries exceeded due to rate limit."}


async def fill_pa_fields_from_summary(referral_summary: str, field_metadata: List[Dict[str, Any]], client) -> Dict[str, List[Dict[str, Any]]]:
    tasks = [
        fill_single_field(field, referral_summary, client)
        for field in field_metadata
    ]
    results = await asyncio.gather(*tasks)
    return {"fields": results}

In [102]:
response = await fill_pa_fields_from_summary(
    referral_summary=summary_text,
    field_metadata=labeled_blocks,
    client=client
)

# Send response["fields"] to your form-filling agent


Rate limit hit. Retrying field 'CB1' in 5.38s (attempt 1/5)...
Rate limit hit. Retrying field 'T3' in 5.06s (attempt 1/5)...
Rate limit hit. Retrying field 'T2' in 5.23s (attempt 1/5)...
Rate limit hit. Retrying field 'T3' in 10.02s (attempt 2/5)...
Rate limit hit. Retrying field 'CB1' in 10.83s (attempt 2/5)...
Rate limit hit. Retrying field 'T2' in 10.35s (attempt 2/5)...
Rate limit hit. Retrying field 'T3' in 20.07s (attempt 3/5)...
Rate limit hit. Retrying field 'T2' in 20.94s (attempt 3/5)...
Rate limit hit. Retrying field 'CB1' in 20.03s (attempt 3/5)...
Rate limit hit. Retrying field 'T16' in 5.65s (attempt 1/5)...
Rate limit hit. Retrying field 'T17' in 5.51s (attempt 1/5)...
Rate limit hit. Retrying field 'T18' in 5.41s (attempt 1/5)...
Rate limit hit. Retrying field 'T17' in 10.20s (attempt 2/5)...
Rate limit hit. Retrying field 'T16' in 10.10s (attempt 2/5)...
Rate limit hit. Retrying field 'T18' in 10.99s (attempt 2/5)...
Rate limit hit. Retrying field 'T16' in 20.82s (atte

In [103]:
response

{'fields': [{'page': 2,
   'name': 'CB1',
   'type': 2,
   'rect': [82.92900085449219,
    110.4580078125,
    93.23079681396484,
    121.0150146484375],
   'label': 'Start of treatment',
   'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n',
   'answer': 'May 22nd'},
  {'page': 2,
   'name': 'T2',
   'type': 7,
   'rect': [210.19700622558594,
    109.281982421875,
    231.8699951171875,
    120.96099853515625],
   'label': 'Start date: (MM)',
   'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n',
   'answer': '05'},
  {'page': 2,
   'name': 'T3',
   'type': 7,
   'rect': [236.83399963378906,
    109.281982421875,
    258.5069885253906,
    120.96099853515625],
   'label': 'Start date: (DD)',
   'block_context': 'Please indicate:\n Start of treatment, start date:  \n Continuation of therapy, date of last treatment: \n',
   'answer': '22'}

In [115]:
from io import BytesIO

In [144]:
def fill_pdf_form_fields(pdf_path: str, field_values: list, output_file=None):
    doc = fitz.open(pdf_path)

    for page in doc:
        for w in page.widgets() or []:
            # Find the matching field dictionary by 'name'
            data = next((f for f in field_values if f["name"] == w.field_name), None)

            if data is None:
                continue

            answer = data.get("answer", "")
            print(f"Filling {w.field_name}: {answer}")

            if w.field_type == fitz.PDF_WIDGET_TYPE_CHECKBOX:
              if (w.field_label == "Yes" and answer == "Yes") or (w.field_label == "No" and answer == "No"):
                  w.field_value = w.field_label  # set to "Yes" or "No"
              else:
                  w.field_value = "Off"
            else:
                w.field_value = str(answer)

            w.update()

    if output_file:
        doc.save(output_file, deflate=True, encryption=fitz.PDF_ENCRYPT_KEEP)
    else:
        from io import BytesIO
        buf = BytesIO()
        doc.save(buf, deflate=True, encryption=fitz.PDF_ENCRYPT_KEEP)
        return buf

    doc.close()



In [145]:
fill_pdf_form_fields("PA.pdf", response['fields'], "filled_form.pdf")

Filling CB1: May 22nd
Filling T2: 05
Filling T3: 22
Filling T4: 2024
Filling CB5: There is no information about prior rituximab or biosimilar treatment.
Filling T6: /
/
Filling T7: 22
Filling T8: There is no specific information about the date of last treatment.
Filling T9: Erfan Rostami, BSN, RN
Filling T10: 615-343-1176
Filling T11: 614-936-0065
Filling T12: Shakh
Filling T13: Abdulla
Filling T14: 04/01/2001
Filling T15: 425 Sherman Ave
Filling T16: Nashville
Filling T17: TN
Filling T18: 37995
Filling T19: 865-395-3958
Filling T20: 614-936-0065
Filling T21: 865-395-0481
Filling T22: This information is not available in the provided document.
Filling T24: This field cannot be filled from the provided context.
Filling T25: Based on the information provided, the patient's Body Surface Area (BSA) is listed as 1.85 m2, which was used to calculate the Rituximab dosage. To calculate the weight in kilograms (kgs) to determine if the BSA is accurate, we would need to use a BSA formula, such a

#Additional Testing

In [ ]:
import fitz  # PyMuPDF
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Open PDF and get dimensions
doc = fitz.open("PA.pdf")
page = doc[1]
page_width = page.mediabox.width
page_height = page.mediabox.height

# Block rectangles (red)
blocks = [
    {'rect': [20.4, 122.12, 524.49, 133.75], 'label': 'patient info header'},
    {'rect': [22.56, 159.03, 372.96, 189.43], 'label': 'rheumatoid arthritis question'},
    {'rect': [22.56, 388.72, 476.93, 420.13], 'label': 'second question'}
]

# Field rectangles (green)
fields = [
    {'rect': [249.62, 167.44, 259.92, 178.0]},
    {'rect': [282.375, 167.44, 292.676, 178.0]},
    {'rect': [334.257, 167.44, 344.558, 178.0]},
    {'rect': [20.80, 131.79, 164.39, 144.46]},
    {'rect': [169.26, 131.79, 348.60, 144.46]},
    {'rect': [354.30, 131.79, 469.83, 144.46]},
    {'rect': [22.71, 419.29, 33.01, 429.85]}
]

# Create plot
fig, ax = plt.subplots(1)
ax.set_xlim(0, page_width)
ax.set_ylim(0, page_height)
ax.invert_yaxis()  # Match PDF coordinate space

# Plot block rectangles
for block in blocks:
    x0, y0, x1, y1 = block['rect']
    ax.add_patch(patches.Rectangle((x0, y0), x1 - x0, y1 - y0, linewidth=1, edgecolor='r', facecolor='none'))
    ax.text(x0, y0, block['label'], fontsize=8, color='blue', verticalalignment='bottom')

# Plot field rectangles
for field in fields:
    x0, y0, x1, y1 = field['rect']
    ax.add_patch(patches.Rectangle((x0, y0), x1 - x0, y1 - y0, linewidth=1, edgecolor='g', facecolor='none'))

# Show plot
plt.title("PDF Field & Block Mapping")
plt.show()